In [1]:
from data_preprocess import *
from data_visual_tools import *
import pandas as pd
import os
import csv
import shutil

In [ ]:
view_labels(
    root="/home/ec2-user/segmenter/ETTDATA/sampled_data_split",
    label_dir='labels/train',
    image_dir='images/train', target_dir='view_label',
    annotation_file='annotations/train_annotations_enl5.json')

In [8]:
f = open('/home/ec2-user/segmenter/MAIDA/data1000/val_annotations_enl5.json')
data = json.load(f)

print(len(data['images']))
print(len(data['annotations']))

166
4414


In [5]:
image_paths = get_img_path(path="/home/ec2-user/efs/MAIDA/normal")
convert_dcm(image_paths, output_path = '/home/ec2-user/segmenter/MAIDA/data1000/new_img')

0 image converted
50 image converted
100 image converted
150 image converted
200 image converted
250 image converted
300 image converted
350 image converted
400 image converted
450 image converted
500 image converted
550 image converted
600 image converted
650 image converted
700 image converted
750 image converted


In [3]:
image_paths = get_img_path(path="/home/ec2-user/efs/MAIDA/abnormal")
convert_dcm(image_paths, output_path = '/home/ec2-user/segmenter/MAIDA/data1000/new_img')

0 image converted
50 image converted
100 image converted
150 image converted
200 image converted
250 image converted
300 image converted
350 image converted
400 image converted
450 image converted
500 image converted
550 image converted


In [4]:
downsize_images(
    root="/home/ec2-user/segmenter/ETTDATA/Test",
    image_dir='RANZCR', target_dir='downsized/RANZCR', output_file='annotations_downsized.json',
    resized_dim=1280)

In [2]:
get_reviewed_anno(ori_ann_file='/home/ec2-user/efs/MAIDA/MIMIC-1105-annotations.json',
                  target_dir='MAIDA/data1000/annotations1105_tmp.json')

In [2]:
generate_gt_label(
    root="/home/ec2-user/segmenter/MAIDA",
    annotation_file='data/ett/annotations/train_annotations_enl5.json',
    image_dir='data1000/downsized/split/images/train', out_dir='data1000/downsized/split/labels/train')

Problem ID:  [4380406, 4146693, 4380560, 4146445, 4379767, 4380433, 2946144, 4379950, 4380559, 4379731, 4380006, 4146328, 4380029, 4380713, 4146035, 4380064, 4380206]


In [35]:
train_info = pd.read_csv('/home/ec2-user/efs/MAIDA/MIMIC-train.csv')
val_info = pd.read_csv('/home/ec2-user/efs/MAIDA/MIMIC-val.csv')
test_info = pd.read_csv('/home/ec2-user/efs/MAIDA/MIMIC-test.csv')

In [2]:
def train_val_test_split(root='/home/ec2-user/segmenter/MAIDA/data1000/downsized',
                         image_dir='images', label_dir='labels', target_dir='downsized/split'):
    src = os.path.join(root, image_dir)
    
    for file in os.listdir(src):
        name = file[:-4]
        if name in list(train_info['imageID']):
            shutil.copy(os.path.join(src, file), os.path.join(root, target_dir, image_dir, "train"))
        elif name in list(val_info['imageID']):
            shutil.copy(os.path.join(src, file), os.path.join(root, target_dir, image_dir, "val"))
        elif name in list(test_info['imageID']):
            shutil.copy(os.path.join(src, file), os.path.join(root, target_dir, image_dir, "test"))
        else:
            print(name)
    src = os.path.join(root, label_dir)
    for file in os.listdir(src):
        name = file[:-4]
        if name in list(train_info['imageID']):
            shutil.copy(os.path.join(src, file), os.path.join(root, target_dir, label_dir, "train"))
        elif name in list(val_info['imageID']):
            shutil.copy(os.path.join(src, file), os.path.join(root, target_dir, label_dir, "val"))
        elif name in list(test_info['imageID']):
            shutil.copy(os.path.join(src, file), os.path.join(root, target_dir, label_dir, "test"))
        else:
            print(name)

In [2]:
def get_test_anno(anno_file='/home/ec2-user/segmenter/MAIDA/data1000/annotations_downsized1105.json',
                  out_file='/home/ec2-user/segmenter/MAIDA/data1000/annotations1105.json',
                  split_info='/home/ec2-user/segmenter/ETT_Evaluation/data_split/train_mimic_only.csv',
                  mimic=False, ranzcr=False, split='test'):
    # column = 'imageID'
    column = 'FileName'
    f = open(anno_file)
    data = json.load(f)
    test_images = []
    info = pd.read_csv(split_info)

    if mimic:
        test_info = info[(info['Split']==split) & (info['Source']=='MIMIC')]
    elif ranzcr:
        test_info = info[(info['Split']==split) & (info['Source']=='RANZCR')]
    else:
        raise ValueError("No dataset specified.")
    for image in data['images']:
        image['file_name'] = image['file_name'].replace('.dcm', '.jpg')
        if image['file_name'][:-4] in list(test_info[column]):
            test_images.append(image)

    test_coco = {
        'info': data['info'],
        'categories': data['categories'],
        'images': test_images,
        'annotations': data['annotations']
    }

    with open(out_file, 'w') as f:
        json.dump(test_coco, f)

In [3]:

def move_ranzcr_images(src_dir='/home/ec2-user/efs/RANZCR/data',
                csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/all_data_split.csv',
                output_dir='/home/ec2-user/segmenter/ETTDATA/Test/RANZCR', split='test', source='RANZCR'):

    os.makedirs(output_dir, exist_ok=True)

    # Loop through the csv file
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Check if the image is in one of the subdirectories
            for subdir_name in os.listdir(src_dir):
                subdir_path = os.path.join(src_dir, subdir_name)
                # print(subdir_path)
                if not os.path.isdir(subdir_path):
                    continue
                img_path = os.path.join(subdir_path, row['FileName'])
                img_path += '.jpg'
                if os.path.isfile(img_path):
                    # If the image is in the csv file and meets the criteria, copy it to the new directory
                    if row['Source'] == source and row['Split'] == split:
                        shutil.copy(img_path, output_dir)


In [4]:
def move_mimic_images(src_dir='/home/ec2-user/segmenter/MAIDA/all_images',
                csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/all_data_split.csv',
                output_dir='/home/ec2-user/segmenter/ETTDATA/Test/MIMIC', split='test', source='MIMIC'):

    os.makedirs(output_dir, exist_ok=True)
    info = pd.read_csv(csv_file)

    target_imgs = list(info[(info['Source']==source) & (info['Split']==split)]['FileName'])

    for img_path in os.listdir(src_dir):
        if img_path.replace('.png', '') in target_imgs:
            image = Image.open(os.path.join(src_dir, img_path))
            img_path = img_path.replace('.png', '.jpg')
            image.save(os.path.join(output_dir, img_path))

In [5]:
# move_ranzcr_images(src_dir='/home/ec2-user/efs/RANZCR/data',
#             csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/sampled_data_split.csv',
#             output_dir='/home/ec2-user/segmenter/ETTDATA/Test/RANZCR', split='test', source='RANZCR')

# move_ranzcr_images(src_dir='/home/ec2-user/efs/RANZCR/data',
#             csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/sampled_data_split.csv',
#             output_dir='/home/ec2-user/segmenter/ETTDATA/sampled_data_split/train', split='train', source='RANZCR')

# move_ranzcr_images(src_dir='/home/ec2-user/efs/RANZCR/data',
#             csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/sampled_data_split.csv',
#             output_dir='/home/ec2-user/segmenter/ETTDATA/sampled_data_split/val', split='val', source='RANZCR')

# move_mimic_images(src_dir='/home/ec2-user/segmenter/MAIDA/all_images',
#                 csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/all_data_split.csv',
#                 output_dir='/home/ec2-user/segmenter/ETTDATA/Test/MIMIC', split='test', source='MIMIC')

move_mimic_images(src_dir='/home/ec2-user/segmenter/MAIDA/all_images',
                csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/train_mimic_only.csv',
                output_dir='/home/ec2-user/segmenter/ETTDATA/mimic_only/train', split='train', source='MIMIC')

move_mimic_images(src_dir='/home/ec2-user/segmenter/MAIDA/all_images',
                csv_file='/home/ec2-user/segmenter/ETT_Evaluation/data_split/train_mimic_only.csv',
                output_dir='/home/ec2-user/segmenter/ETTDATA/mimic_only/val', split='val', source='MIMIC')


In [11]:
# downsize_anno(ori_ann_file='/home/ec2-user/efs/RANZCR/RANZCR-ETT-annotations.json',
#     output_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/ranzcr-downsized.json',
#     resized_dim=1280)
# make_bbox_nonzero(root='/home/ec2-user/segmenter/ETTDATA/downsized_anno',
#                   anno_dir='ranzcr-downsized.json')
# enlarge_bbox(root="/home/ec2-user/segmenter/ETTDATA/downsized_anno",
#              annotation_file='ranzcr-downsized.json',
#              out_file='ranzcr-enlarged5.json', factor=5)
# create_area(file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/ranzcr-enlarged5.json')
# get_test_anno(anno_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/ranzcr-enlarged5.json',
#               out_file='/home/ec2-user/segmenter/ETTDATA/Test/downsized/RANZCR/annotations/test_annotations_enl5.json',
#               split_info='/home/ec2-user/segmenter/ETT_Evaluation/data_split/train_mimic_only.csv',
#               mimic=False, ranzcr=True)
# downsize_images(root="/home/ec2-user/segmenter/ETTDATA/Test",
#     image_dir='RANZCR', target_dir='downsized/RANZCR/images',
#     resized_dim=1280)



# downsize_anno(ori_ann_file='/home/ec2-user/segmenter/ETT_Evaluation/MIMIC-ETT-annotations-cleaned.json',
#     output_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/mimic-downsized.json',
#     resized_dim=1280)
# make_bbox_nonzero(root='/home/ec2-user/segmenter/ETTDATA/downsized_anno',
#                   anno_dir='mimic-downsized.json')
# enlarge_bbox(root="/home/ec2-user/segmenter/ETTDATA/downsized_anno",
#              annotation_file='mimic-downsized.json',
#              out_file='mimic-enlarged5.json', factor=5)
# create_area(file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/mimic-enlarged5.json')
get_test_anno(anno_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/mimic-enlarged5.json',
              out_file='/home/ec2-user/segmenter/ETTDATA/Test/downsized/MIMIC/annotations/test_annotations_enl5.json',
              split_info='/home/ec2-user/segmenter/ETT_Evaluation/data_split/train_mimic_only.csv',
              mimic=True, ranzcr=False)
# downsize_images(root="/home/ec2-user/segmenter/ETTDATA/Test",
#     image_dir='MIMIC', target_dir='downsized/MIMIC/images',
#     resized_dim=1280)


In [10]:

# get_test_anno(anno_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/mimic-enlarged5.json',
#               out_file='/home/ec2-user/segmenter/ETTDATA/train_mimic_only/annotations/train_mimic_annotations_enl5.json',
#               split_info='/home/ec2-user/segmenter/ETT_Evaluation/data_split/train_mimic_only.csv',
#               mimic=True, ranzcr=False, split='train')
# get_test_anno(anno_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/mimic-enlarged5.json',
#               out_file='/home/ec2-user/segmenter/ETTDATA/train_mimic_only/annotations/val_mimic_annotations_enl5.json',
#               split_info='/home/ec2-user/segmenter/ETT_Evaluation/data_split/train_mimic_only.csv',
#               mimic=True, ranzcr=False, split='val')

# get_test_anno(anno_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/ranzcr-enlarged5.json',
#               out_file='/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/train_ranzcr_annotations_enl5.json',
#               split_info='/home/ec2-user/segmenter/ETT_Evaluation/data_split/sampled_data_split.csv',
#               mimic=False, ranzcr=True, split='train')
# get_test_anno(anno_file='/home/ec2-user/segmenter/ETTDATA/downsized_anno/ranzcr-enlarged5.json',
#               out_file='/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/val_ranzcr_annotations_enl5.json',
#               split_info='/home/ec2-user/segmenter/ETT_Evaluation/data_split/sampled_data_split.csv',
#               mimic=False, ranzcr=True, split='val')

In [12]:
def combine_anno(anno1, anno2, output_anno):
    with open(anno1) as f1:
        data1 = json.load(f1)
    with open(anno2) as f2:
        data2 = json.load(f2)

    data = {}
    data['categories'] = data1['categories'] # same for mimic and ranzcr
    data['images'] = data1['images']
    data['images'].extend(data2['images'])

    data['annotations'] = data1['annotations']
    data['annotations'].extend(data2['annotations'])

    with open(output_anno, 'w') as f:
        json.dump(data, f)


In [13]:
# combine_anno('/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/val_mimic_annotations_enl5.json',
#              '/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/val_ranzcr_annotations_enl5.json',
#              '/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/val_annotations_enl5.json')
# combine_anno('/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/train_mimic_annotations_enl5.json',
#              '/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/train_ranzcr_annotations_enl5.json',
#              '/home/ec2-user/segmenter/ETTDATA/sampled_data_split/annotations/train_annotations_enl5.json')

downsize_images(root="/home/ec2-user/segmenter/ETTDATA/sampled_data_split",
    image_dir='val', target_dir='val',
    resized_dim=1280)

downsize_images(root="/home/ec2-user/segmenter/ETTDATA/sampled_data_split",
    image_dir='train', target_dir='train',
    resized_dim=1280)

In [3]:
from data_preprocess import *
generate_gt_label(
    root="/home/ec2-user/segmenter/ETTDATA/train_mimic_only/",
    annotation_file='annotations/val_annotations_enl5.json',
    image_dir='images/val', out_dir='labels/val')
generate_gt_label(
    root="/home/ec2-user/segmenter/ETTDATA/train_mimic_only/",
    annotation_file='annotations/train_annotations_enl5.json',
    image_dir='images/train', out_dir='labels/train')

Problem ID:  [4146445, 2946144, 4380406]
Problem ID:  [4146693, 4146035, 4146193, 4380560, 4379767, 4380433, 4380559, 4380006, 4380029, 4380713, 4380064, 4380206]


In [2]:
get_stats(root="/home/ec2-user/segmenter/MAIDA/data1000", image_dir='downsized/split/images/train')

Mean of the dataset:  [126.55846604 126.55846604 126.55846604]
Std of the dataset:  [55.47551373 55.47551373 55.47551373]


(array([126.55846604, 126.55846604, 126.55846604]),
 array([55.47551373, 55.47551373, 55.47551373]))

In [4]:
def train_val_test_split_anno(
                         anno_dir='/home/ec2-user/segmenter/MAIDA/data1000/annotations_downsized1105.json',
                         out_dir='/home/ec2-user/segmenter/MAIDA/data1000'):
    f = open(anno_dir)
    data = json.load(f)
    train_images = []
    val_images = []
    test_images = []
    for image in data['images']:
        image['file_name'] = image['file_name'].replace('.dcm', '.png')
        if image['file_name'][:-4] in list(train_info['imageID']):
            train_images.append(image)
        elif image['file_name'][:-4] in list(val_info['imageID']):
            val_images.append(image)
        elif image['file_name'][:-4] in list(test_info['imageID']):
            test_images.append(image)
        else: 
            print(image)
    train_coco = {
        'info': data['info'],
        'categories': data['categories'],
        'images': train_images,
        'annotations': data['annotations']
    }

    val_coco = {
        'info': data['info'],
        'categories': data['categories'],
        'images': val_images,
        'annotations': data['annotations']
    }

    test_coco = {
        'info': data['info'],
        'categories': data['categories'],
        'images': test_images,
        'annotations': data['annotations']
    }

    with open(os.path.join(out_dir, 'train_annotations.json'), 'w') as f:
        json.dump(train_coco, f)

    with open(os.path.join(out_dir, 'val_annotations.json'), 'w') as f:
        json.dump(val_coco, f)

    with open(os.path.join(out_dir, 'test_annotations.json'), 'w') as f:
        json.dump(test_coco, f)

In [ ]:
train_val_test_split_anno(anno_dir='/home/ec2-user/segmenter/ETT_Evaluation/MIMIC-ETT-annotations-cleaned.json',
            out_dir='/home/ec2-user/segmenter/ETTDATA/Test/annotations/RANZCR'
)

In [17]:
make_bbox_nonzero()

Zero boxes: 0


In [5]:
# f = open('/home/ec2-user/segmenter/MAIDA/data1000/annotations1105.json')
# data = json.load(f)
# for ann in data['images']:
#     ann['file_name'] = ann['file_name'].replace('.dcm', '.png')
# with open('/home/ec2-user/segmenter/MAIDA/data1000/annotations1105.json', 'w') as f:
#     json.dump(data, f)

In [18]:
downsize_anno(ori_ann_file='MIMIC-2k-annotations.json',
    root="/home/ec2-user/segmenter/MAIDA",
    output_file='annotations-downsized-2k.json',
    resized_dim=1280)

In [19]:
make_bbox_nonzero(root='/home/ec2-user/segmenter/MAIDA',
                         anno_dir='annotations-downsized-2k.json')

Zero boxes: 0


In [20]:
enlarge_bbox(root="/home/ec2-user/segmenter/MAIDA",
             annotation_file='annotations-downsized-2k.json', factor=5)
# generate_gt_label(
#     root="/home/ec2-user/segmenter/MAIDA/data1000",
#     annotation_file='annotations_enlarged_5.json',
#     image_dir='downsized/images', out_dir='downsized/labels')
# train_val_test_split_anno(root='/home/ec2-user/segmenter/MAIDA/data1000',
#                          anno_dir='annotations_enlarged_5.json')
# train_val_test_split(root='/home/ec2-user/segmenter/MAIDA/data1000/downsized',
#                          image_dir='images', label_dir='labels', target_dir='split')

In [34]:
make4classes(file='/home/ec2-user/segmenter/MAIDA/data1000/val_annotations.json',
        outfile='/home/ec2-user/segmenter/MAIDA/data1000/val_annotations_4cls.json')

In [25]:
file='/home/ec2-user/segmenter/MAIDA/data1000/annotations1105_tmp.json'
m = {}
rep = []
with open(file) as f:
    data = json.load(f)

for ann in data['annotations']:
    key = (ann['image_id'], ann['category_id'])
    if key not in m.keys():
        m[key] = 0
    m[key] += 1
    # elif ann['category_id'] == 3046 or ann['category_id'] == 3047:
    #     prev = m[key]
    #     # if ann['bbox'] == prev['bbox'] and ann['assignee'] == prev['assignee']:
    #     #     data['annotations'].remove(ann)
    #     # else:
    #     rep.append(ann)


In [23]:
create_area(file='/home/ec2-user/segmenter/MAIDA/annotations_enlarged_5.json')

In [48]:
f_2k = open('/home/ec2-user/segmenter/MAIDA/annotations_enlarged_5.json')
f_train = open('/home/ec2-user/segmenter/MAIDA/data/ett/annotations/train_annotations_enl5.json')
f_val = open('/home/ec2-user/segmenter/MAIDA/data/ett/annotations/val_annotations_enl5.json')
f_test = open('/home/ec2-user/segmenter/MAIDA/data/ett/annotations/test_annotations_enl5.json')

data_2k = json.load(f_2k)
data_train = json.load(f_train)
data_val = json.load(f_val)
data_test = json.load(f_test)

In [50]:
val_ids = [i['id'] for i in data_val['images']]
test_ids = [i['id'] for i in data_test['images']]
train_ids = [i['id'] for i in data_train['images']]

In [52]:
# rep_ids = [i['image_id'] for i in rep]
# print(len(rep_ids))
cnt = 0
tmp = 0
for ann in data_2k['images']:
    if ann['id'] not in val_ids and ann['id'] not in test_ids and ann['id'] not in train_ids:
        data_train['images'].append(ann)
        tmp += 1
    else:
        cnt += 1
data_train['annotations'] = data_2k['annotations']


In [55]:
cnt, tmp

(1105, 1336)

In [56]:
len(data_train['images'])

2109

In [57]:
with open('/home/ec2-user/segmenter/MAIDA/data/ett/annotations/train_annotations_enl5.json', 'w') as outfile:
    json.dump(data_train, outfile)

In [61]:
f = open('/home/ec2-user/segmenter/MAIDA/data/ett/annotations/train_annotations_enl5.json')
data_train = json.load(f)

for ann in data_train['images']:
    ann['file_name'] = ann['file_name'].replace('.dcm', '.png')

with open('/home/ec2-user/segmenter/MAIDA/data/ett/annotations/train_annotations_enl5.json', 'w') as outfile:
    json.dump(data_train, outfile)